In [121]:
from pymongo import MongoClient
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd

In [95]:
client = MongoClient()
db = client['capstone-project']

In [96]:
db.collection_names()

['system.indexes',
 'nyt-recipe-metadata',
 'nyt-recipe-html',
 'nyt-recipe-extracted']

In [97]:
tab = db['nyt-recipe-extracted']

In [98]:
tab.find_one().keys()

dict_keys(['_id', 'categories', 'ingredients', 'ingredient_names', 'directions', 'web_url', 'description', 'author', 'name'])

In [99]:
params = {'_id': 0, 'ingredient_names': 1, 'name': 1, 'web_url': 1}

result = tab.find({}, params)

ingredients = []
names = []
urls = []

for recipe in result:
    ingredients.append(';'.join(recipe['ingredient_names']))
    names.append(recipe['name'])
    urls.append(recipe['web_url'])

In [131]:
result = tab.find({}, params)
df = pd.DataFrame(columns=['ingredients', 'names', 'urls'])

for ix, recipe in enumerate(result):

    df = pd.concat([df, pd.DataFrame({'ingredients': ';'.join(recipe['ingredient_names']),
                                     'names': recipe['name'],
                                     'urls': recipe['web_url']}, index=[ix,])]
                  )

In [132]:
df.head()

,ingredients,names,urls
0,all-purpose flour;kosher salt;cold unsalted bu...,Cheese Wafers or Straws,http://cooking.nytimes.com/recipes/1016355-che...
1,all-purpose flour;granulated sugar;dark brown ...,Blackberry Crumb Muffins,http://cooking.nytimes.com/recipes/1014979-bla...
2,blueberries;sugar;lemon juice;cinnamon,Blueberry Jam With Lime,http://cooking.nytimes.com/recipes/1014975-blu...
3,eggplants;olive oil;Salt and pepper;garlic;cay...,Smoky Eggplant Soup,http://cooking.nytimes.com/recipes/1014970-smo...
4,freshly squeezed orange juice;New Mexico chile...,Grilled Chile Flank Steak With Salsa,http://cooking.nytimes.com/recipes/1014966-gri...


In [100]:
ingredients[1]

'all-purpose flour;granulated sugar;dark brown sugar;cinnamon;ground allspice;fine sea salt;unsalted butter;unsalted butter;granulated sugar;eggs;vanilla extract;lemon;all-purpose flour;baking powder;baking soda;fine sea salt;fresh blackberries'

In [101]:
names[1]

'Blackberry Crumb Muffins'

In [102]:
urls[1]

'http://cooking.nytimes.com/recipes/1014979-blackberry-crumb-muffins'

In [103]:
def tokenizer(x):
    return x.lower().split(';')

cv = CountVectorizer(analyzer='word', tokenizer=tokenizer)
wc = cv.fit_transform(ingredients)

In [114]:
search_term = 'kale'
search_term in cv.vocabulary_

True

In [115]:
ix = cv.vocabulary_[search_term]
print(ix)

6468


In [116]:
match = wc[:, ix].nonzero()[0]

In [117]:
name_url = zip(np.array(names)[match], np.array(urls)[match])

In [118]:
for n,u in name_url:
    print('{}: {}'.format(n,u))

Ribollita: http://cooking.nytimes.com/recipes/1016052-ribollita
Kale and Red Cabbage Slaw With Walnuts: http://cooking.nytimes.com/recipes/1015779-kale-and-red-cabbage-slaw-with-walnuts
Mixed Grains Risotto With Kale, Walnuts and Black Quinoa: http://cooking.nytimes.com/recipes/1015776-mixed-grains-risotto-with-kale-walnuts-and-black-quinoa
Leek, Kale and Potato Latkes: http://cooking.nytimes.com/recipes/1015736-leek-kale-and-potato-latkes
Grilled Scallops With Kale and Olives: http://cooking.nytimes.com/recipes/1014824-grilled-scallops-with-kale-and-olives
Spanish Tortilla with Mushrooms and Kale: http://cooking.nytimes.com/recipes/1017357-spanish-tortilla-with-mushrooms-and-kale
Blood Orange Smoothie With Grapes and Red Quinoa: http://cooking.nytimes.com/recipes/1017297-blood-orange-smoothie-with-grapes-and-red-quinoa
Quinoa and Rice Bowl With Kale, Kimchi and Egg: http://cooking.nytimes.com/recipes/1016855-quinoa-and-rice-bowl-with-kale-kimchi-and-egg
Grilled Striped Bass With Charr